<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2004%20-%20Intro_to_Prompt_Engineering_Tips_and_Tricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install required packages
!pip install -q langchain==0.3.26 langchain-openai==0.3.23 langchain-core==0.3.66 langchain-community==0.3.26 openai==1.92.0

In [5]:
import os

# os.environ["OPENAI_API_KEY"]="OPENAI_API_KEY"

from google.colab import userdata
os.environ["OPENAI_API_KEY"]= userdata.get('OPENAI_API_KEY')

# Story Generation

In [6]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

prompt_system = "You are a helpful assistant whose goal is to help write stories."

prompt = """Continue the following story. Write no more than 200 words.

Once upon a time, in a world where animals could speak, a courageous \
mouse named Benjamin decided to"""

response = client.responses.create(
    model="gpt-4.1-mini",
    instructions = prompt_system,
    input= prompt,
    max_output_tokens=200,
    temperature=0.7
)

print("Story continuation:")
print(response.output_text)

Story continuation:
Once upon a time, in a world where animals could speak, a courageous mouse named Benjamin decided to embark on a daring adventure beyond the safety of his home. For years, he had listened to tales of the enchanted forest, where ancient trees whispered secrets and magical creatures roamed freely. Though small in size, Benjamin’s heart was filled with bravery and curiosity.

One crisp morning, with a tiny satchel packed with cheese and a map drawn by an old owl, Benjamin set off. As he ventured deeper into the forest, he encountered a wise fox named Alaric, who warned him of the lurking shadows that guarded the forest’s greatest secret—a glowing crystal said to grant immense wisdom.

Determined, Benjamin pressed on, navigating tangled roots and shimmering streams. Along the way, he befriended a chatty squirrel and a graceful deer, who joined him on his quest. Together, they faced challenges that tested their courage and friendship.

At last, in a clearing bathed in mo

# Product Description

In [7]:
product_prompt_system = """You are a helpful assistant whose goal is to help write \
product descriptions."""

product_prompt = """Write a captivating product description for a luxurious, handcrafted\
, limited-edition fountain pen made from rosewood and gold.
Write no more than 100 words."""

response = client.responses.create(
    model="gpt-4.1-mini",
    instructions = product_prompt_system,
    input= product_prompt,
    max_output_tokens=100,
)

print("Product Description:")
print(response.output_text)

Product Description:
Experience timeless elegance with our handcrafted, limited-edition fountain pen, meticulously crafted from rich rosewood and gleaming gold. Each pen is a masterpiece, combining exquisite artistry with unparalleled writing precision. The smooth, flowing nib delivers an effortless writing experience, while the warm, natural hues of rosewood contrast beautifully with the luxurious gold accents. Perfect for collectors and connoisseurs, this exclusive pen embodies sophistication and exclusivity, making every word you write a statement of style and refinement. Elevate your writing ritual


# Zero-Shot Prompting

In [8]:
zero_shot_prompt_system = "You are a helpful assistant whose goal is to write short poems."


topic = "summer"

# Format the prompt with the topic
zero_shot_prompt = f"Write a short poem about {topic}."

response = client.responses.create(
    model="gpt-4.1-mini",
    instructions=zero_shot_prompt_system,
    input=zero_shot_prompt,
    max_output_tokens=200,
)

print("Zero-shot poem about summer:")
print(response.output_text)

Zero-shot poem about summer:
Golden sun on skies so blue,  
Whispers warm, the whole day through.  
Laughter dances, breezes play,  
Summer’s magic lights our way.


# In-Context Learning

In [9]:
prompt_system = "You are a helpful assistant whose goal is to write short poems."

prompt = """Write a short poem about {topic}."""

examples = {
    "nature": """Birdsong fills the air,\nMountains high and valleys \
deep,\nNature's music sweet.""",
    "winter": """Snow blankets the ground,\nSilence is the only \
sound,\nWinter's beauty found."""
}

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt.format(topic="nature")},
        {"role": "assistant", "content": examples["nature"]},
        {"role": "user", "content": prompt.format(topic="winter")},
        {"role": "assistant", "content": examples["winter"]},
        {"role": "user", "content": prompt.format(topic="summer")}
    ]
)

print("In-context learning poem about summer:")
print(response.output_text)

In-context learning poem about summer:
Golden rays cascade,  
Laughter dances in warm breeze,  
Summer’s vibrant glow.


# Few Shot Prompting

In [10]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Examples for few-shot learning
examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

# Template for formatting each example
example_formatter_template = """
Color: {color}
Emotion: {emotion}
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

# Few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:",
    suffix="Now, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="",
)

# Chain using LCEL (LangChain Expression Language)
chain = few_shot_prompt | llm | StrOutputParser()

# Run the chain with the input
response = chain.invoke({"input": "purple"})

print( response.strip())

Color: purple  
Emotion: creativity


# Role Prompting

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Prompt template
template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""

prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
)

# Chain using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

# Run the chain to get the AI-generated song title
response = chain.invoke(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response.strip())


Theme: interstellar travel
Year: 3030
AI-generated song title: How about **"Quantum Drift: Voyage Beyond the Stars"**? It captures the futuristic vibe and the essence of interstellar travel in the year 3030!


## Chain Prompting

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Chain for the first prompt
chain_question = prompt_question | llm | StrOutputParser()

# Run the chain for the first prompt with an empty dictionary
response_question = chain_question.invoke({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Chain for the second prompt
chain_fact = prompt_fact | llm | StrOutputParser()

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the chain for the second prompt
response_fact = chain_fact.invoke(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Answer: Albert Einstein
Fact: Albert Einstein's theory of general relativity is a fundamental theory of gravitation that describes gravity not as a force, but as a curvature of spacetime caused by the presence of mass and energy. It explains how massive objects like stars and planets warp the fabric of spacetime, influencing the motion of other objects and the path of light. This theory has been confirmed by numerous experiments and observations and is essential for understanding phenomena such as black holes, gravitational waves, and the expansion of the universe.


# Bad Prompt Practices

In [13]:
template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=["topic"],
    template=template,
)
prompt.format(topic="dogs")

'Tell me something about dogs.'

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something interesting about {scientist}.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the chain for the first prompt
chain_question = prompt_question | llm | StrOutputParser()

# Run the chain for the first prompt with an empty dictionary
response_question = chain_question.invoke({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the chain for the second prompt
chain_fact = prompt_fact | llm | StrOutputParser()

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the chain for the second prompt
response_fact = chain_fact.invoke(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Answer: Albert Einstein
Fact: Albert Einstein, one of the most famous physicists in history, developed the theory of relativity, which revolutionized our understanding of space, time, and gravity. Interestingly, despite his groundbreaking work in physics, Einstein was also a passionate advocate for civil rights and was offered the presidency of Israel in 1952, which he declined.


In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Prompt 1
template_question = """What are some musical genres?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details.
Answer: """
prompt_fact = PromptTemplate(input_variables=["genre1", "genre2", "genre3"], template=template_fact)

# Create the chain for the first prompt
chain_question = prompt_question | llm | StrOutputParser()

# Run the chain for the first prompt with an empty dictionary
response_question = chain_question.invoke({})

print(response_question)

# Assign three hardcoded genres
genre1, genre2, genre3 = "jazz", "pop", "rock"

# Create the chain for the second prompt
chain_fact = prompt_fact | llm | StrOutputParser()

# Input data for the second prompt
input_data = {"genre1": genre1, "genre2": genre2, "genre3": genre3}

# Run the chain for the second prompt
response_fact = chain_fact.invoke(input_data)

print("Genres:", genre1, genre2, genre3)
print("Fact:", response_fact)

Some musical genres include:

- Pop
- Rock
- Jazz
- Classical
- Hip Hop
- Country
- Blues
- Electronic
- Reggae
- Metal
- R&B (Rhythm and Blues)
- Folk
- Punk
- Soul
- Disco

Each genre has its own unique style, history, and cultural significance.
Genres: jazz pop rock
Fact: Jazz, pop, and rock are distinct musical styles that each offer unique expressions and cultural influences. They vary in rhythm, instrumentation, and emotional tone, reflecting different artistic approaches and audience experiences. Each genre has contributed significantly to the evolution of modern music and continues to inspire creativity across the world.


# Tips for Effective Prompting

In [16]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the chain for the few-shot prompt template
chain = few_shot_prompt_template | llm | StrOutputParser()

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the chain for the user query
response = chain.invoke({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response: To improve communication skills, focus on active listening, practice clear and concise expression, and be mindful of nonverbal cues. Additionally, seek feedback and engage in regular conversations to build confidence.


In [17]:
# Bad Prompt Practices

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# BAD EXAMPLE 1: Vague and context-less
template_vague = "Analyze this and give me insights about {data}."
prompt_vague = PromptTemplate(
    input_variables=["data"],
    template=template_vague,
)

# BAD EXAMPLE 2: Conflicting instructions
template_conflict = """Write a detailed comprehensive analysis of {topic} but keep it brief and don't include too much information. Be specific but also general."""
prompt_conflict = PromptTemplate(
    input_variables=["topic"],
    template=template_conflict,
)

# BAD EXAMPLE 3: No output format guidance
template_no_format = "Tell me about {company} financial performance."
prompt_no_format = PromptTemplate(
    input_variables=["company"],
    template=template_no_format,
)

# BAD EXAMPLE 4: Overly complex single prompt
template_complex = """Given {product}, analyze the market, competitors, pricing strategy, customer segments, marketing channels, potential risks, growth opportunities, financial projections, and implementation timeline while also considering regulatory requirements and technological trends."""
prompt_complex = PromptTemplate(
    input_variables=["product"],
    template=template_complex,
)

# BAD EXAMPLE 5: Assumptions without context
template_assumptions = "Fix the obvious issues with {code} and optimize it."
prompt_assumptions = PromptTemplate(
    input_variables=["code"],
    template=template_assumptions,
)

# Example of running bad prompts
chain_vague = prompt_vague | llm | StrOutputParser()
response_vague = chain_vague.invoke({"data": "sales numbers"})

print("Vague prompt result:")
print(response_vague)

chain_no_format = prompt_no_format | llm | StrOutputParser()
response_no_format = chain_no_format.invoke({"company": "Apple"})

print("\nNo format guidance result:")
print(response_no_format)

Vague prompt result:
Sure! Please provide the sales data or the document you'd like me to analyze, and I'll give you insights based on the sales numbers.

No format guidance result:
As of my knowledge cutoff in June 2024, here is an overview of Apple Inc.'s recent financial performance:

### Revenue and Profitability
Apple has consistently demonstrated strong financial performance, driven by its diverse product lineup including the iPhone, iPad, Mac, Apple Watch, and services such as the App Store, Apple Music, and iCloud. In its fiscal year 2023, Apple reported:

- **Revenue:** Approximately $383 billion, marking a steady increase compared to previous years.
- **Net Income:** Around $100 billion, reflecting strong profitability and efficient cost management.
- **Gross Margin:** Typically around 40-42%, indicating healthy margins on its products and services.

### Key Drivers
- **iPhone Sales:** The iPhone remains Apple's largest revenue contributor, with the latest models continuing t